In [1]:
from mxnet import autograd, nd, init, gluon
from mxnet.gluon import nn, data as gdata, loss as gloss
import numpy as np

## 生成数据集

In [2]:
num_inputs = 3
num_examples = 10000
true_w = [1.1, -2.2, 3.3]
true_b = 4.4
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_w[2] * features[:, 2] + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)
print('features：', features)
print('labels：', labels)

features： 
[[ 1.1630785e+00  4.8380461e-01  2.9956347e-01]
 [ 1.5302546e-01 -1.1688148e+00  1.5580710e+00]
 [-5.4594457e-01 -2.3556297e+00  5.4144025e-01]
 ...
 [-6.7329638e-05  2.0219166e+00 -1.0645875e+00]
 [ 1.8973123e-01 -1.5727270e+00  1.3594624e-01]
 [-1.6451789e+00  5.9507078e-01 -1.5699839e-02]]
<NDArray 10000x3 @cpu(0)>
labels： 
[ 5.62155   12.283315  10.751223  ... -3.5522704  8.511643   1.2163162]
<NDArray 10000 @cpu(0)>


## 读取数据集

In [3]:
batch_size = 50
dataset = gdata.ArrayDataset(features, labels)
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)
for x, y in data_iter:
    print(x, y)
    break


[[-0.5767784  -0.4428551  -1.2591642 ]
 [ 0.767156    0.00393082  0.7827675 ]
 [ 1.3861587   0.13081668  0.48218653]
 [-0.41712606  0.80658615  2.4631624 ]
 [ 0.2165102  -2.4998713   0.4777952 ]
 [ 0.14601442  1.5703346   1.6983985 ]
 [-0.0636088   1.1851114  -0.33775288]
 [ 0.9161404  -0.0743828   1.2854688 ]
 [-0.00597976 -0.00504663  0.88081807]
 [ 1.1010166   2.2547026   0.927516  ]
 [ 0.15117064 -0.56956995 -1.4035162 ]
 [ 1.0352751  -0.06155001 -0.7781916 ]
 [-0.7944512   0.92204165 -0.30174804]
 [-0.01882095 -0.74916095  0.44509676]
 [ 0.36922586 -1.7841098  -1.0389587 ]
 [ 0.9197123  -1.1944344  -0.5842977 ]
 [ 1.1334809   0.49158767 -0.36942866]
 [ 0.72617674 -0.02762904  0.36120465]
 [ 0.3014626   1.1198691  -1.288111  ]
 [-0.14374009 -0.9276457  -1.0578752 ]
 [-0.27754712 -1.3157445   0.86454093]
 [ 0.01912635  0.42826512 -0.6306099 ]
 [ 0.5111985   0.2900958   0.60595375]
 [-0.5596014  -0.25511396 -0.96646553]
 [-0.18522854 -2.01737     0.4669863 ]
 [ 2.5810065  -1.1698867

In [4]:
net = nn.Sequential()
net.add(nn.Dense(1))

net.initialize(init.Normal(sigma=0.01))

loss = gloss.L2Loss()

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.01})

## 训练模型

In [5]:
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    for x, y in data_iter:
        with autograd.record():
            l = loss(net(x), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

epoch 1, loss: 0.323812
epoch 2, loss: 0.005820
epoch 3, loss: 0.000153
epoch 4, loss: 0.000052
epoch 5, loss: 0.000050
epoch 6, loss: 0.000050
epoch 7, loss: 0.000050
epoch 8, loss: 0.000050
epoch 9, loss: 0.000050
epoch 10, loss: 0.000050


In [6]:
dense = net[0]
print('true_w：', [true_w])
print('train_w：', dense.weight.data().asnumpy())
print('true_b：', [true_b])
print('train_b：', dense.bias.data().asnumpy())

true_w： [[1.1, -2.2, 3.3]]
train_w： [[ 1.1002226 -2.1999476  3.2999659]]
true_b： [4.4]
train_b： [4.399887]
